#Fingerprinter.ipynb v1.2

#What is this?
This is a computer program that takes the "fingerprint" of a text file that you give it. The purpose is to make it easier to spot tampering. A tiny change in the text will cause the fingerprint to be obviously different.

#How do I use this on a desktop?
1. You are probably reading this on the `colab.research.google.com` website or the `github.com` website. In the former case, skip directly to step 2. In the latter case, if you are unfamiliar with a webpage like this, the easiest thing to do is click this link and then proceed to step 2:
https://colab.research.google.com/drive/1nPrdk4w_sl6zjJVWP-DS7HKIyuIth6KN?usp=sharing
If that doesn't work, the next-easiest thing to do is signup for Google Colab, perhaps do one of the tutorials, paste the code below into a blank notebook, and then proceed to step 2. (Also note - the above link is for the most recent version; see below for previous versions.)
2. Save the document that you want to fingerprint as a .txt file - for example in Google Docs, go to File > Download > Plain text (.txt)
3. Upload your .txt file by clicking the folder icon to the left, then the upload icon.  
4. Hover your mouse just to the left of the cell containing "!pip install..." You should see a "play" icon. Click this. Some setup happens automatically for a couple of minutes.
5. Do the same thing in the next cell. Nothing will happen except you should see a little green check mark in a few seconds. This is more automatic setup.
6. In the next cell, set `input_path` to the name of the file you uploaded.
7. Now click the play icon to the left of the cell you just edited.
8. You should see a message containing the fingerprint.
9. Copy-paste that output message into the end of your document.
10. If you make revisions to your document, don't worry about removing the fingerprint before repeating the above steps - however, the result of this script will change, so you'll need to overwrite the old fingerprint with the updated one.

#What about a mobile device?
1. It is basically the same as above, but more awkward. Chrome on an ipad should work ok.

#Is it perfect?
No. Certain text changes will not cause the fingerprint to change; this is on purpose, because sometimes web browsers make small changes when copy-pasting text between various apps, and we do not want those to confuse us. This script ignores the numbers in numbered lists. It ignores the tilde character and various other special characters. So don't use this to compare source code. Plus whatever bugs I'm unaware of. But as long as we run the same version of the script, we'll get the same answer for the same input.

#What if I want to run my text through an earlier version of this program?
Excellent question! We shouldn't be surprised if different versions (e.g. Fingerprinter.ipynb version 1.0 versus 1.1) give different fingerprints for the same input text. This means that if you try to verify a policy's fingerprint and it does not match the fingerprint it claims it has, or it does not match the fingerprint of the corresponding draft, one of the first things to check is that you are using the same version of Fingerprinter.ipynb that was used to make the fingerprint being claimed at the end of the document or in the vote log. Here is a list of all versions so far:  
- Fingerprinter.ipynb v1.2  [Github](https://github.com/dspumpkins/fingerprinter/blob/main/Fingerprinter.ipynb)  [Colab](https://colab.research.google.com/drive/1nPrdk4w_sl6zjJVWP-DS7HKIyuIth6KN?usp=sharing)  
- Fingerprinter.ipynb v1.1+mc  [Github](https://github.com/dspumpkins/fingerprinter/blob/90dd1b0a5580eb7898d3b15927d43ddf57dcc275/Fingerprinter.ipynb)  [Colab](https://drive.google.com/file/d/1hJwDabjtxTQ5FIZTq1avAz4-rWg6vbqw/view?usp=sharing)  
- Fingerprinter.ipynb v1.1  [Github](https://github.com/dspumpkins/fingerprinter/blob/e062a8953ea0777a5f7a86fe82e2a972d2af72e6/Fingerprinter.ipynb)  [Colab](https://drive.google.com/file/d/1fxOgCxXkyNsKQwyEgPuqlhpBfuZ09kBg/view?usp=sharing)  
- Fingerprinter.ipynb v1.0  [Github](https://github.com/dspumpkins/fingerprinter/blob/a6670449f00f4afe4fba8069e9161f99979befd7/Fingerprinter.ipynb)  [Colab](https://drive.google.com/file/d/1_qoxvHHOBE7icUf4cpGSnETILHRp_-ik/view?usp=sharing)  

(note: v1.1+mc has nearly identical code to v1.1. The sole difference is I added 'mc' - more commentary.)

#This code is doing lots of substitutions of special characters. What if I have math equations, e.g. an accounting formula, in my document?
In that case it is better to make the equation in something like Microsoft Equation Editor, take a screenshot, upload it to a draft of your reddit post, then download it (reddit does some image processing that usually changes the fingerprint of whatever image you upload), then use this script to take the hash of all those equation images, and add those hashes to your main document as image captions using the reddit post editor. See the `process_directory` function in the code below.

In [ ]:
!pip install python-docx odfpy beautifulsoup4 pillow striprtf

In [14]:
import os
import sys
import hashlib
import re
from pathlib import Path
from docx import Document
from lxml import etree
from odf.opendocument import load as load_odt
from odf.text import P, H, Span, List, ListItem, Note, NoteBody
from odf.element import Text
from bs4 import BeautifulSoup
from PIL import Image
from striprtf.striprtf import rtf_to_text

VERSION = "1.2"

# Function to read a txt file
def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8-sig') as file:
        text = file.read()
    return text

# Function to normalize text
def normalize_text(text):
    def not_whitespace_related(text):
        # Remove item numbers and letters from numbered and lettered lists
        text = re.sub(r'^\s*[\d]+\.\s*|^\s*[a-zA-Z]\.\s*', '', text, flags=re.MULTILINE)

        # Remove asterisks signifying unordered list items
        text = re.sub(r'^\s*\*\s+', '', text, flags=re.MULTILINE)

        # Remove double-asterisks
        text = re.sub(r'\*\*', '', text, flags=re.MULTILINE)

        # Remove tildes
        text = re.sub(r'\~', '', text, flags=re.MULTILINE)

        # convert '/r/subreddit' to 'r/subreddit' (unless it is the last part of a URL)
        text = re.sub(r'(?<!\w)/r/(?![^/\w]*$)', 'r/', text)

        # Remove leading and trailing whitespace
        text = text.strip()

        # Replace bracketed footnote numbers with 'neat' footnote numbers
        text = re.sub(r'\[(\d+)\]', r'\1', text)

        # Remove lines that consist solely of underscores
        text = re.sub(r'^_+$', '', text, flags=re.MULTILINE)

        # Define ligature replacements
        ligature_map = {
            'ﬁ': 'fi', 'ﬂ': 'fl', 'ﬃ': 'ffi', 'ﬄ': 'ffl', 'ﬀ': 'ff', 'æ': 'ae', 'œ': 'oe'
        }
        for ligature, replacement in ligature_map.items():
            text = text.replace(ligature, replacement)

        # Define confusable symbol replacements
        confusables_map = {
            '“': '"', '”': '"', '‘': "'", '’': "'", '–': '-', '—': '-', '…': '...',
            ' ': ' ', '−': '-', '×': 'x', '·': '.', '«': '"', '»': '"', '„': '"',
            '©': '(C)', '®': '(R)', '™': '(TM)'
        }
        for symbol, replacement in confusables_map.items():
            text = text.replace(symbol, replacement)
        return text

    repeat_flag = True
    while repeat_flag:
        #print("cranking...")
        hash_before = compute_sha256(text)
        # try to remove all special characters
        text = not_whitespace_related(text)
        hash_after = compute_sha256(text)
        repeat_flag = hash_before != hash_after
    #print("done cranking")

    # Remove all whitespaces, newlines, carriage returns
    text = re.sub(r'\s+', '', text)

    return text


# Function to compute SHA-256 checksum
def compute_sha256(data):
    sha256 = hashlib.sha256()
    sha256.update(data.encode('utf-8'))
    return sha256.hexdigest()

# Function to process input file
def process_file(file_path):
    ext = file_path.suffix.lower()
    if ext == '.txt':
        text = read_txt(file_path)
    else:
        raise ValueError(f"Unsupported file format: {ext}")

    #print(f"### Printing `text` ###")
    #print(text)
    #print("### Done printing `text` ###")

    # Split the text to ignore the message and below
    start_ignore_marker = "---------(Fingerprinter script should ignore the following!)-----------"
    stop_ignore_marker = "---------(Stop ignoring now)-----------"

    parts = text.split(start_ignore_marker, 1)
    if len(parts) > 1:
        #print(f"### Printing `parts[0]` ###")
        #print(parts[0])
        #print("### Done printing `parts[0]` ###")
        #print(f"### Printing `parts[1]` ###")
        #print(parts[1])
        #print("### Done printing `parts[1]` ###")

        parts_ignored = parts[1].split(stop_ignore_marker, 1)
        if len(parts_ignored) > 1:
            text_to_normalize = parts[0] + parts_ignored[1]
        else:
            text_to_normalize = parts[0]
    else:
        text_to_normalize = text

    #print(text_to_normalize)
    #print("### Done printing `text_to_normalize`###")

    normalized_text = normalize_text(text_to_normalize)

    #print(f"### Printing `normalized_text` ###")
    #print(normalized_text)
    #print("### Done printing `normalized_text`###")

    checksum = compute_sha256(normalized_text)
    message = f"""---------(Fingerprinter script should ignore the following!)-----------
The fingerprint (SHA-256 checksum) of this document's text content (with whitespace, special characters, and this message removed) is:
{checksum}
(Generated by Fingerprinter.ipynb version {VERSION})
https://github.com/dspumpkins/fingerprinter/blob/main/Fingerprinter.ipynb
---------(Stop ignoring now)-----------"""
    print(message)


# Function to process directory of .webp files
def process_directory(directory_path):
    for file_path in directory_path.glob('*.webp'):
        with open(file_path, 'rb') as file:
            data = file.read()
            checksum = hashlib.sha256(data).hexdigest()
            print(f"SHA-256 checksum for {file_path}: {checksum}")



In [ ]:
input_path = "Replace me with the name of your file.txt"


input_path = Path(input_path)
if input_path.is_file():
    process_file(input_path)
elif input_path.is_dir():
    process_directory(input_path)
else:
    raise ValueError(f"Invalid input path: {input_path}")